In [1]:
import numpy as np
from ase import Atoms
from ase.build import bulk
from ase.visualize import view
from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.functional import Functional, TotalFunctional, FunctionalOutput
from dftpy.functional.abstract_functional import AbstractFunctional
from dftpy.ions import Ions
from dftpy.constants import Units
from dftpy.formats import io
from dftpy.mpi import mp
from ase.io import read
from dftpy.math_utils import ecut2nr
from dftpy.optimization import Optimization


path_pp='/Users/valeria/Documents/PP/ofpp/EAC/upf/blps/'
file='si.lda.upf'
PP_list = {'Si': path_pp+file}

In [2]:
import os
os.chdir('../..')
path_file = os.getcwd()

In [3]:
dictionary = {
    'Si_Btin': {'file': 'Si_Btin.vasp'}, 
    'Si_fcc': {'file': 'Si_fcc.vasp'},
    'Si_bcc': {'file': 'Si_bcc.vasp'},
    'Si_cd': {'file': 'Si_cd.vasp'},
    'Si_dhcp': {'file': 'Si_dhcp.vasp'},
    'Si_bct5': {'file': 'Si_bct5.vasp'},
    'Si_sh': {'file': 'Si_sh.vasp'},
    'Si_cbcc': {'file': 'Si_cbcc.vasp'},
    'Si_hcp': {'file': 'Si_hcp.vasp'},
}

def get_rho_ini(phase, charge):
    inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
    ions = Ions.from_ase(read(inputfile, format='vasp'))
    fu_ks = io.read_density(path_file+'/Results/Fukui/'+str(phase)+'_fukui.xsf')
    nr = fu_ks.grid.nr
#     print(phase, nr)
    grid = DirectGrid(lattice=ions.cell, nr=nr)
    if charge is None:
        pseudo = Functional(type='PSEUDO', grid = grid, ions=ions, PP_list=PP_list)
        rho_ini = DirectField(grid=grid)
        rho_ini[:] = ions.get_ncharges()/ions.cell.volume
    else:
        pseudo = Functional(type='PSEUDO', grid = grid, ions=ions, PP_list=PP_list)        
        rho_ini = DirectField(grid=grid)
        rho_ini[:] = (ions.get_ncharges()-charge)/ions.cell.volume
    return rho_ini, grid, pseudo, ions

def get_rho(pseudo, rho_ini, rho0):
    ke = Functional(type='KEDF',name='WT', rho0= rho0)
    evaluator = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=pseudo)
    optimization_options = {'econv' : 1e-7}
    opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
            optimization_method = 'TN')
    Rho_opt = opt.optimize_rho(guess_rho=rho_ini)
    return Rho_opt

def fukui_fun(rho_n, rho_dn):
    return (rho_dn - rho_n)/0.1

In [4]:
Phases = ['Si_Btin', 'Si_fcc', 'Si_bcc', 'Si_cd', 'Si_dhcp', 'Si_hcp', 'Si_bct5', 'Si_sh']   
# Phases = ['Si_bcc']
Rho_ini = []
Grid = []
Pseudo = []
IONS = []

Rho_ini_charged = []
Grid_charged = []
Pseudo_charged = []
IONS_charged = [] 
for i,phase in enumerate(Phases):
    rho_ini, grid, pseudo, ions = get_rho_ini(phase, charge=None)
    Rho_ini.append(rho_ini)
    Grid.append(grid)
    Pseudo.append(pseudo)
    IONS.append(ions)

    rho_ini, grid, pseudo, ions = get_rho_ini(phase, charge=0.1)
    Rho_ini_charged.append(rho_ini)
    Grid_charged.append(grid)
    Pseudo_charged.append(pseudo)
    IONS_charged.append(ions)    

setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key:

In [5]:
import h5py
file = h5py.File(path_file+"/organize_data/Data_phases.hdf5", "r")
fileX = file["Descriptor"]
rho0_KE_WT = file["rho0_KE_wt"]
rho0_TE_WT = file["rho0_TE_wt"]
rho0_DEN_WT = file["rho0_DEN_wt"]

rho0_KE_wt =[]
rho0_TE_wt =[]
rho0_DEN_wt=[]
rho = []
for i in np.arange(0,280):
    rho0_KE_wt.append(rho0_KE_WT[i][0]) 
    rho0_TE_wt.append(rho0_TE_WT[i][0]) 
    rho0_DEN_wt.append(rho0_DEN_WT[i][0])
file.close()
rho0_Btin_KE_wt = rho0_KE_wt[0:30];rho0_fcc_KE_wt = rho0_KE_wt[30:60];rho0_bcc_KE_wt = rho0_KE_wt[60:90];rho0_cd_KE_wt = rho0_KE_wt[90:120];rho0_dhcp_KE_wt = rho0_KE_wt[120:150];rho0_Btin_hd_KE_wt = rho0_KE_wt[150:180];rho0_rand4_KE_wt = rho0_KE_wt[180:185];rho0_rand8_KE_wt = rho0_KE_wt[185:190];rho0_bct5_KE_wt = rho0_KE_wt[190:220];rho0_hcp_KE_wt = rho0_KE_wt[220:250];rho0_sh_KE_wt = rho0_KE_wt[250:280]

rho0_Btin_TE_wt = rho0_TE_wt[0:30];rho0_fcc_TE_wt = rho0_TE_wt[30:60];rho0_bcc_TE_wt = rho0_TE_wt[60:90];rho0_cd_TE_wt = rho0_TE_wt[90:120];rho0_dhcp_TE_wt = rho0_TE_wt[120:150];rho0_Btin_hd_TE_wt = rho0_TE_wt[150:180];rho0_rand4_TE_wt = rho0_TE_wt[180:185];rho0_rand8_TE_wt = rho0_TE_wt[185:190];rho0_bct5_TE_wt = rho0_TE_wt[190:220];rho0_hcp_TE_wt = rho0_TE_wt[220:250];rho0_sh_TE_wt = rho0_TE_wt[250:280]

rho0_Btin_DEN_wt = rho0_DEN_wt[0:30];rho0_fcc_DEN_wt = rho0_DEN_wt[30:60];rho0_bcc_DEN_wt = rho0_DEN_wt[60:90];rho0_cd_DEN_wt = rho0_DEN_wt[90:120];rho0_dhcp_DEN_wt = rho0_DEN_wt[120:150];rho0_Btin_hd_DEN_wt = rho0_DEN_wt[150:180];rho0_rand4_DEN_wt = rho0_DEN_wt[180:185];rho0_rand8_DEN_wt = rho0_DEN_wt[185:190];rho0_bct5_DEN_wt = rho0_DEN_wt[190:220];rho0_hcp_DEN_wt = rho0_DEN_wt[220:250];rho0_sh_DEN_wt = rho0_DEN_wt[250:280]

In [6]:
rho0_fcc_TE_wt= [0.09673084,0.09021482,0.08449455,0.07952551,0.0751474 ,0.07119835,0.06768235,0.06452084,0.06163877,0.0590039 ,0.05640183,0.05364946,0.05005879,0.04647901,0.04368379,0.04189186,0.04072559,0.03974306,0.03875802,0.0377161 ,0.03674104,0.03577425,0.03483485,0.03391843,0.03304381,0.0321998 ,0.03140109,0.03063086,0.02989303,0.02919675]

# rho0_bct5_DEN_wt = [0.06900083,0.06255148,0.05745174,0.05293614,0.04895279,0.04262975,0.0389589 ,0.03619192,0.03375811,0.03177499,0.03022887,0.02891853,0.02840704,0.02788642,0.02790464,0.02908319,0.03084631,0.03255065,0.03191184,0.03371278,0.03336661,0.03390128,0.03446173,0.03473932,0.03516789,0.03531672,0.03486211,0.03582237,0.03545685,0.03604892]

rho0_bct5_TE_wt = [0.06533168, 0.05630981, 0.04473504, 0.03978841, 0.03651514,0.03423538, 0.03246512, 0.03103024, 0.02983059, 0.02882025,0.02796016, 0.02722429, 0.02656433, 0.02595726, 0.02544661,0.02502034, 0.02466189, 0.02436471, 0.02411465, 0.02389834,0.02370005, 0.02352027, 0.02334777, 0.02317923, 0.02301507,0.02285681, 0.02270007, 0.02254006, 0.02238082, 0.02221971]

rho0_bct5_KE_wt = np.load("/Users/valeria/Documents/aiWT/Final_version/wt/wt/ks_calculations/Si_bct5/DATA2/rho0_dT.npy")

In [7]:
rho0_TE = [rho0_Btin_TE_wt, rho0_fcc_TE_wt, rho0_bcc_TE_wt, rho0_cd_TE_wt, rho0_dhcp_TE_wt, rho0_hcp_TE_wt, rho0_bct5_TE_wt, rho0_sh_TE_wt]
rho0_KE = [rho0_Btin_KE_wt, rho0_fcc_KE_wt, rho0_bcc_KE_wt, rho0_cd_KE_wt, rho0_dhcp_KE_wt, rho0_hcp_KE_wt, rho0_bct5_KE_wt, rho0_sh_KE_wt]
rho0_DEN = [rho0_Btin_DEN_wt, rho0_fcc_DEN_wt, rho0_bcc_DEN_wt, rho0_cd_DEN_wt, rho0_dhcp_DEN_wt, rho0_hcp_DEN_wt, rho0_bct5_DEN_wt, rho0_sh_DEN_wt]

In [8]:
xc = Functional(type='XC',name='LDA')
hartree = Functional(type='HARTREE')
rho_opt_KE = []
rho_opt_TE = []
rho_opt_DEN = []
rho_opt_AVE = []
rho_opt_charged_KE = []
rho_opt_charged_TE = []
rho_opt_charged_DEN = []
rho_opt_charged_AVE = []
for i,phase in enumerate(Phases):
    rho_opt_KE.append(get_rho(Pseudo[i], Rho_ini[i], rho0_KE[i][10]))
    rho_opt_TE.append(get_rho(Pseudo[i], Rho_ini[i], rho0_TE[i][10]))
    rho_opt_DEN.append(get_rho(Pseudo[i], Rho_ini[i], rho0_DEN[i][10]))
    rho_opt_AVE.append(get_rho(Pseudo[i], Rho_ini[i], Rho_ini[i].mean()))
    
    rho_opt_charged_KE.append(get_rho(Pseudo[i], Rho_ini_charged[i], rho0_KE[i][10]))
    rho_opt_charged_TE.append(get_rho(Pseudo[i], Rho_ini_charged[i], rho0_TE[i][10]))
    rho_opt_charged_DEN.append(get_rho(Pseudo[i], Rho_ini_charged[i], rho0_DEN[i][10]))
    rho_opt_charged_AVE.append(get_rho(Pseudo[i], Rho_ini_charged[i], Rho_ini[i].mean()))

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.516417282922E+01     -1.516417E+01   2.736092E+00    1       1       4.031324E-02    
1       -1.607625668266E+01     -9.120839E-01   5.899505E-01    12      2       2.801521E-01    
2       -1.608988346878E+01     -1.362679E-02   3.770396E-02    8       1       3.497543E-01    
3       -1.609173173007E+01     -1.848261E-03   1.735383E-03    12      1       4.476690E-01    
4       -1.609183765753E+01     -1.059275E-04   1.371958E-04    15      1       5.646081E-01    
5       -1.609184415905E+01     -6.501524E-06   1.263229E-05    12      1       6.663470E-01    
6       -1.609184521513E+01     -1.056079E-06   1.253043E-06    16      1       7.949262E-01    
7       -1.609184525783E+01     -4.270000E-08   1.193569E-07    9       1       8.712931E-01    
8       -1.609184526422E+01     -6.392433E-09   1.059138E-08    15      1       9.849832E-01    
#### Density Optimization Conv

3       -1.610842569718E+01     -1.611939E-03   1.437636E-03    12      1       2.748771E-01    
4       -1.610850713065E+01     -8.143347E-05   9.413497E-05    15      1       3.934827E-01    
5       -1.610851170100E+01     -4.570352E-06   8.865443E-06    12      1       4.867520E-01    
6       -1.610851239914E+01     -6.981389E-07   7.892355E-07    16      1       6.131978E-01    
7       -1.610851242538E+01     -2.623843E-08   7.338419E-08    9       1       6.943870E-01    
8       -1.610851242936E+01     -3.976105E-09   6.365828E-09    15      1       8.153229E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.3421530385007011
Chemical potential (eV)  : 9.310458410052766
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.526117300627E+01     -1.526117E+01   3.674299E+00    1       1       1.408672E-02    
1       -1.590713418979E+01     -6.459612E-01   3.252539E-01    2       1       4.626679E-02

8       -1.601524744420E+01     -7.006754E-09   1.639025E-08    11      1       4.466119E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.613393098887023
Chemical potential (eV)  : 16.691276398497653
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.532720945234E+01     -1.532721E+01   3.651334E+00    1       1       5.905867E-03    
1       -1.596838936256E+01     -6.411799E-01   3.176305E-01    2       1       2.255082E-02    
2       -1.601187477520E+01     -4.348541E-02   2.435570E-02    15      1       1.039748E-01    
3       -1.601286242655E+01     -9.876513E-04   2.167526E-03    10      1       1.583400E-01    
4       -1.601298227135E+01     -1.198448E-04   1.982365E-04    13      1       2.264669E-01    
5       -1.601298937503E+01     -7.103678E-06   1.945775E-05    9       1       2.777231E-01    
6       -1.601299041476E+01     -1.039734E-06   1.489439E-06    12      1       3.418138E-01

Chemical potential (eV)  : 10.082927110587903
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -7.738816498852E+00     -7.738816E+00   1.280679E+00    1       1       5.071878E-03    
1       -8.065123648881E+00     -3.263072E-01   2.119057E-01    12      2       6.527376E-02    
2       -8.071538714561E+00     -6.415066E-03   1.701490E-02    9       1       1.086028E-01    
3       -8.072422553617E+00     -8.838391E-04   1.200455E-03    13      1       1.727450E-01    
4       -8.072505774659E+00     -8.322104E-05   8.879149E-05    15      1       2.438998E-01    
5       -8.072510791140E+00     -5.016481E-06   1.411111E-05    12      1       2.977798E-01    
6       -8.072511314818E+00     -5.236777E-07   1.714942E-06    12      1       3.533659E-01    
7       -8.072511409956E+00     -9.513823E-08   1.211229E-07    15      1       4.189389E-01    
8       -8.072511413640E+00     -3.684002E-09   8.040409E-09    10      1       4

2       -3.229286499068E+01     -2.308231E+00   1.052918E+00    19      2       8.131340E-01    
3       -3.231449469027E+01     -2.162970E-02   1.010238E-01    7       1       9.821460E-01    
4       -3.231940199714E+01     -4.907307E-03   8.448174E-03    15      1       1.325817E+00    
5       -3.231998984542E+01     -5.878483E-04   3.057493E-03    12      1       1.597158E+00    
6       -3.232012766401E+01     -1.378186E-04   4.144623E-04    16      1       1.959546E+00    
7       -3.232016618055E+01     -3.851655E-05   1.757476E-04    13      1       2.248259E+00    
8       -3.232018626016E+01     -2.007960E-05   4.538055E-05    16      1       2.592715E+00    
9       -3.232018722604E+01     -9.658866E-07   2.978163E-06    6       1       2.746160E+00    
10      -3.232018738973E+01     -1.636895E-07   1.917648E-07    19      1       3.160764E+00    
11      -3.232018739622E+01     -6.486566E-09   1.594744E-08    9       1       3.366580E+00    
12      -3.232018739891E+01   

...

3       -1.605320924125E+01     -6.714921E-02   4.945151E-01    15      1       4.996238E-01    
4       -1.607144660636E+01     -1.823737E-02   7.821242E-02    17      1       7.063169E-01    
5       -1.607685127577E+01     -5.404669E-03   4.696696E-02    12      1       8.374629E-01    
6       -1.607891016293E+01     -2.058887E-03   1.088323E-02    14      1       9.870257E-01    
7       -1.607894508073E+01     -3.491780E-05   7.074328E-03    13      1       1.130325E+00    
8       -1.608049829087E+01     -1.553210E-03   3.852885E-04    23      6       1.412593E+00    
9       -1.608050338143E+01     -5.090554E-06   3.588204E-05    7       1       1.495599E+00    
10      -1.608050402308E+01     -6.416475E-07   1.877444E-05    13      1       1.632761E+00    
11      -1.608050604794E+01     -2.024862E-06   2.345318E-06    19      4       1.859216E+00    
12      -1.608050609017E+01     -4.223434E-08   2.261787E-07    11      1       1.981166E+00    
13      -1.608050609197E+01   

#### Density Optimization Converged ####
Chemical potential (a.u.): 0.4075977697608538
Chemical potential (eV)  : 11.091300255633755
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -7.752847746105E+00     -7.752848E+00   1.336800E+00    1       1       5.785704E-03    
1       -8.027702103138E+00     -2.748544E-01   2.445819E-01    14      1       7.221198E-02    
2       -8.043125904780E+00     -1.542380E-02   2.028653E-01    12      1       1.294620E-01    
3       -8.068394398537E+00     -2.526849E-02   1.494992E-01    12      2       1.905401E-01    
4       -8.072265649441E+00     -3.871251E-03   1.401709E-02    10      1       2.385328E-01    
5       -8.073223657412E+00     -9.580080E-04   5.968281E-03    13      1       2.970769E-01    
6       -8.073405551585E+00     -1.818942E-04   5.624420E-04    11      1       3.462708E-01    
7       -8.073456324502E+00     -5.077292E-05   9.910031E-05    16      1       4.147546E-0

#### Density Optimization Converged ####
Chemical potential (a.u.): 0.28358486813650885
Chemical potential (eV)  : 7.716737317531824
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.458846062175E+01     -1.458846E+01   3.204624E+00    1       1       9.311199E-03    
1       -1.603852767739E+01     -1.450067E+00   1.011081E+00    13      2       1.300409E-01    
2       -1.605895466557E+01     -2.042699E-02   7.762639E-02    7       1       1.943169E-01    
3       -1.606349759662E+01     -4.542931E-03   9.860464E-03    13      1       3.095210E-01    
4       -1.606404920302E+01     -5.516064E-04   2.201376E-03    13      1       4.262638E-01    
5       -1.606422585506E+01     -1.766520E-04   3.529842E-04    16      1       5.693190E-01    
6       -1.606423874131E+01     -1.288625E-05   3.138127E-05    11      1       6.745520E-01    
7       -1.606424123835E+01     -2.497038E-06   2.185020E-06    16      1       8.279929E-0

6       -4.023000252529E+00     -8.833490E-07   1.410897E-05    12      1       2.136919E-01    
7       -4.023001350754E+00     -1.098225E-06   5.642283E-07    19      2       2.657180E-01    
8       -4.023001359959E+00     -9.205085E-09   4.403946E-08    9       1       2.890499E-01    
9       -4.023001362035E+00     -2.075837E-09   3.380708E-09    15      1       3.289979E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.3654628438263747
Chemical potential (eV)  : 9.94475052092252
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -3.801072869396E+00     -3.801073E+00   7.011073E-01    1       1       3.857851E-03    
1       -4.018887516721E+00     -2.178146E-01   1.844172E-01    14      2       4.574704E-02    
2       -4.022118172050E+00     -3.230655E-03   1.790278E-02    9       1       6.997204E-02    
3       -4.022762715056E+00     -6.445430E-04   1.511733E-03    14      1       1.065569E-01 

1       -4.051548997128E+00     -2.087359E-01   1.657369E-01    14      2       4.246616E-02    
2       -4.054427304743E+00     -2.878308E-03   1.571453E-02    9       1       6.650019E-02    
3       -4.054950022023E+00     -5.227173E-04   3.462408E-03    12      1       9.757495E-02    
4       -4.055023248839E+00     -7.322682E-05   3.125067E-04    15      1       1.357310E-01    
5       -4.055030543039E+00     -7.294201E-06   3.622822E-05    12      1       1.659260E-01    
6       -4.055031555540E+00     -1.012501E-06   1.481132E-05    13      1       1.992650E-01    
7       -4.055032715279E+00     -1.159739E-06   1.338480E-06    17      2       2.474859E-01    
8       -4.055032739678E+00     -2.439918E-08   1.001218E-07    8       1       2.692921E-01    
9       -4.055032743922E+00     -4.244233E-09   1.597497E-08    13      1       3.010449E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.36061184030357674
Chemical potential (eV)  : 9.8127479914

In [9]:
fu_of_ks_KE  = {}
fu_of_ks_TE  = {}
fu_of_ks_DEN = {}
fu_of_ks_AVE = {}
hartree_fukui_KE  = {}
hartree_fukui_TE  = {}
hartree_fukui_DEN = {}
hartree_fukui_AVE = {}

for i, p in enumerate(Phases):
    fu_KE = -fukui_fun(rho_opt_KE[i], rho_opt_charged_KE[i])-1
    fu_TE = -fukui_fun(rho_opt_TE[i], rho_opt_charged_TE[i])-1
    fu_DEN = -fukui_fun(rho_opt_DEN[i], rho_opt_charged_DEN[i])-1
    fu_AVE = -fukui_fun(rho_opt_AVE[i], rho_opt_charged_AVE[i])-1
    
#     fu_KE = - (rho_opt_charged_KE[i]-rho_opt_KE[i])/0.1
    
    fukui_ks = io.read_density(path_file+'/Results/Fukui/'+str(p)+'_fukui.xsf')

    fu_of_ks_KE[p] =(1/2 * (np.abs(fu_KE-fukui_ks)).integral())
    fu_of_ks_TE[p] =(1/2 * (np.abs(fu_TE-fukui_ks)).integral())
    fu_of_ks_DEN[p]=(1/2 * (np.abs(fu_DEN-fukui_ks)).integral())
    fu_of_ks_AVE[p]=(1/2 * (np.abs(fu_AVE-fukui_ks)).integral())
    
    fg_KE   = fu_KE.fft()
    fg_TE   = fu_TE.fft()
    fg_DEN  = fu_DEN.fft()
    fg_AVE  = fu_AVE.fft()

    invg_KE   = fg_KE.grid.gg
    invg_TE   = fg_TE.grid.gg
    invg_DEN  = fg_DEN.grid.gg
    invg_AVE  = fg_AVE.grid.gg
    
    mask_KE   = invg_KE!=0
    mask_TE   = invg_TE!=0
    mask_DEN  = invg_DEN!=0
    mask_AVE  = invg_AVE!=0

    invg_KE[mask_KE ] = 1/invg_KE[mask_KE]
    invg_TE[mask_TE ] = 1/invg_TE[mask_TE]
    invg_DEN[mask_DEN] = 1/invg_DEN[mask_DEN]
    invg_AVE[mask_AVE] = 1/invg_AVE[mask_AVE]

    fu_ha_KE   = ((fg_KE.imag**2+fg_KE.real**2)*4*np.pi*invg_KE).integral()
    fu_ha_TE   = ((fg_TE.imag**2+fg_TE.real**2)*4*np.pi*invg_TE).integral()
    fu_ha_DEN  = ((fg_DEN.imag**2+fg_DEN.real**2)*4*np.pi*invg_DEN).integral()
    fu_ha_AVE  = ((fg_AVE.imag**2+fg_AVE.real**2)*4*np.pi*invg_AVE).integral()

    hartree_fukui_KE[p] = fu_ha_KE
    hartree_fukui_TE[p] = fu_ha_TE
    hartree_fukui_DEN[p] = fu_ha_DEN
    hartree_fukui_AVE[p] = fu_ha_AVE

    print(p, hartree_fukui_KE[p])
    print(hartree(fu_KE).energy)


Si_Btin 0.013787840173399608
0.0005889094655617296
Si_fcc 0.011087459422723133
0.00015296696954842523
Si_bcc 0.02346049863731891
0.0005108849565952334
Si_cd 0.005333434740761442
0.0008038769182896762
Si_dhcp 0.010990390672549132
0.0015858949443090331
Si_hcp 0.022337728964544185
0.00043651299273909477
Si_bct5 0.012705610231278372
0.0010162574909508358
Si_sh 0.05531373922716886
0.0021461324486964353


In [10]:
fu_ke = {}
fu_te = {}
fu_den = {}
for i, p in enumerate(Phases):
    fu_ke[p]  = fu_of_ks_KE[p]
    fu_te[p]  = fu_of_ks_TE[p]
    fu_den[p] = fu_of_ks_DEN[p]

In [43]:
from tabulate import tabulate
## Energies wrt KS bcc energies
data = [
    ["Btin", fu_ke['Si_Btin'], fu_te['Si_Btin'], fu_den['Si_Btin']],
    ["bcc",  fu_ke['Si_bcc'], fu_te['Si_bcc'], fu_den['Si_bcc']],
    ["fcc",  fu_ke['Si_fcc'], fu_te['Si_fcc'], fu_den['Si_fcc']],
    ["hcp",  fu_ke['Si_hcp'], fu_te['Si_hcp'], fu_den['Si_hcp']],
    ["bct5", fu_ke['Si_bct5'], fu_te['Si_bct5'], fu_den['Si_bct5']],
    ["sh",   fu_ke['Si_sh'], fu_te['Si_sh'], fu_den['Si_sh']],
    ["cd",   fu_ke['Si_cd'], fu_te['Si_cd'], fu_den['Si_cd']],
    ["dhcp", fu_ke['Si_dhcp'], fu_te['Si_dhcp'], fu_den['Si_dhcp']],
]

table = tabulate(data, headers=[r"Fukui function"," rho0_KIN"," rho0_TE"," rho0_DEN", " N/V", "KS"], tablefmt="grid")

print(table)
# print(tabulate(data, tablefmt="latex"))

+------------------+-------------+------------+-------------+
| Fukui function   |    rho0_KIN |    rho0_TE |    rho0_DEN |
+==================+=============+============+=============+
| Btin             |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| bcc              |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| fcc              |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| hcp              |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| bct5             |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| sh               |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| cd               |           1 |          1 |           1 |
+-------

In [12]:
import pickle
with open('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Fukui/Fukui_hartree3.pkl', 'rb') as f:
    ks_fh = pickle.load(f)
    

In [13]:
fu_ha_ke  = {}
fu_ha_te  = {}
fu_ha_den = {}
fu_ha_ave = {}
for i, p in enumerate(Phases):
    fu_ha_ke[p]  = hartree_fukui_KE[p]*100
    fu_ha_te[p]  = hartree_fukui_TE[p]*100
    fu_ha_den[p] = hartree_fukui_DEN[p]*100
    fu_ha_ave[p] = hartree_fukui_AVE[p]*100

In [14]:
from tabulate import tabulate
## Energies wrt KS bcc energies
data = [
    ["Btin", fu_ha_ke['Si_Btin'], fu_ha_te['Si_Btin'], fu_ha_den['Si_Btin'],  fu_ha_ave['Si_Btin'],  ks_fh['Si_Btin']*100 ],
    ["bcc",  fu_ha_ke['Si_bcc'],  fu_ha_te['Si_bcc'],  fu_ha_den['Si_bcc'], fu_ha_ave['Si_bcc'],   ks_fh['Si_bcc']*100 ],
    ["fcc",  fu_ha_ke['Si_fcc'],  fu_ha_te['Si_fcc'],   fu_ha_den['Si_fcc'], fu_ha_ave['Si_fcc'], ks_fh['Si_fcc']*100 ],
    ["hcp",  fu_ha_ke['Si_hcp'],  fu_ha_te['Si_hcp'],  fu_ha_den['Si_hcp'],  fu_ha_ave['Si_hcp'],  ks_fh['Si_hcp']*100 ],
    ["bct5", fu_ha_ke['Si_bct5'], fu_ha_te['Si_bct5'], fu_ha_den['Si_bct5'],   fu_ha_ave['Si_bct5'],   ks_fh['Si_bct5']*100 ],
    ["sh",   fu_ha_ke['Si_sh'],   fu_ha_te['Si_sh'], fu_ha_den['Si_sh'], fu_ha_ave['Si_sh'],  ks_fh['Si_sh']*100 ],
    ["cd",   fu_ha_ke['Si_cd'],   fu_ha_te['Si_cd'],  fu_ha_den['Si_cd'],  fu_ha_ave['Si_cd'], ks_fh['Si_cd']*100 ],
    ["dhcp", fu_ha_ke['Si_dhcp'], fu_ha_te['Si_dhcp'],   fu_ha_den['Si_dhcp'],   fu_ha_ave['Si_dhcp'], ks_fh['Si_dhcp']*100 ],
]

table = tabulate(data, headers=[r"Fukui function"," rho0_KIN"," rho0_TE"," rho0_DEN", " N/V", "KS"], tablefmt="grid")

print(table)
print(tabulate(data, tablefmt="latex"))

+------------------+-------------+------------+-------------+----------+----------+
| Fukui function   |    rho0_KIN |    rho0_TE |    rho0_DEN |      N/V |       KS |
+==================+=============+============+=============+==========+==========+
| Btin             |    1.37878  |   1.37412  |    1.34284  | 1.29812  | 1.28694  |
+------------------+-------------+------------+-------------+----------+----------+
| bcc              |    2.34605  |   2.33747  |    2.40101  | 2.41522  | 1.83535  |
+------------------+-------------+------------+-------------+----------+----------+
| fcc              |    1.10875  |   1.10849  |    1.10935  | 1.10349  | 0.990101 |
+------------------+-------------+------------+-------------+----------+----------+
| hcp              |    2.23377  |   2.23283  |    2.21383  | 2.25015  | 2.19007  |
+------------------+-------------+------------+-------------+----------+----------+
| bct5             |    1.27056  |   1.25134  |    1.23543  | 1.14274  | 1.2

In [45]:
from tabulate import tabulate
## Energies wrt KS bcc energies
data = [
    ["Btin", fu_ha_ke['Si_Btin'], fu_ha_te['Si_Btin'], fu_ha_den['Si_Btin'],  fu_ha_ave['Si_Btin'],  ks_fh['Si_Btin']*100 ],
    ["bcc",  fu_ha_ke['Si_bcc'],  fu_ha_te['Si_bcc'],  fu_ha_den['Si_bcc'], fu_ha_ave['Si_bcc'],   ks_fh['Si_bcc']*100 ],
    ["fcc",  fu_ha_ke['Si_fcc'],  fu_ha_te['Si_fcc'],   fu_ha_den['Si_fcc'], fu_ha_ave['Si_fcc'], ks_fh['Si_fcc']*100 ],
    ["hcp",  fu_ha_ke['Si_hcp'],  fu_ha_te['Si_hcp'],  fu_ha_den['Si_hcp'],  fu_ha_ave['Si_hcp'],  ks_fh['Si_hcp']*100 ],
    ["bct5", fu_ha_ke['Si_bct5'], fu_ha_te['Si_bct5'], fu_ha_den['Si_bct5'],   fu_ha_ave['Si_bct5'],   ks_fh['Si_bct5']*100 ],
    ["sh",   fu_ha_ke['Si_sh'],   fu_ha_te['Si_sh'], fu_ha_den['Si_sh'], fu_ha_ave['Si_sh'],  ks_fh['Si_sh']*100 ],
    ["cd",   fu_ha_ke['Si_cd'],   fu_ha_te['Si_cd'],  fu_ha_den['Si_cd'],  fu_ha_ave['Si_cd'], ks_fh['Si_cd']*100 ],
    ["dhcp", fu_ha_ke['Si_dhcp'], fu_ha_te['Si_dhcp'],   fu_ha_den['Si_dhcp'],   fu_ha_ave['Si_dhcp'], ks_fh['Si_dhcp']*100 ],
]

table = tabulate(data, headers=[r"Fukui function"," rho0_KIN"," rho0_TE"," rho0_DEN", " N/V", "KS"], tablefmt="grid")

print(table)
print(tabulate(data, tablefmt="latex"))

+------------------+-------------+------------+-------------+----------+----------+
| Fukui function   |    rho0_KIN |    rho0_TE |    rho0_DEN |      N/V |       KS |
+==================+=============+============+=============+==========+==========+
| Btin             |    1.34368  |   1.33766  |    1.298    | 1.29813  | 1.28694  |
+------------------+-------------+------------+-------------+----------+----------+
| bcc              |    2.25242  |   2.24481  |    2.38372  | 2.41519  | 1.83535  |
+------------------+-------------+------------+-------------+----------+----------+
| fcc              |    1.10648  |   1.10223  |    1.10499  | 1.10305  | 0.990101 |
+------------------+-------------+------------+-------------+----------+----------+
| hcp              |    2.20653  |   2.19819  |    2.19219  | 2.25014  | 2.19007  |
+------------------+-------------+------------+-------------+----------+----------+
| bct5             |    1.19833  |   1.19884  |    1.21228  | 1.14273  | 1.2